# EDA Project: Medical Appointment No-Show

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 

> This dataset collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment. A number of characteristics about the patient are included in each row.
Features are:
>> <b>PatientId</b>: the ID of the patient, mostly I will delete this column because we do not need it.
>> <br><b>AppointmentId</b>: the ID of the appointment, mostly I will delete this column because we do not need it.
>><br><b>Gender</b>: Male or Female.
>><br><b>ScheduledDay</b>: on what day the patient set up their appointment.
>><br><b>AppointmentDay</b>: it is the appointment day
>><br><b>Age</b>: How old is the patient.
>><br><b>Neighborhood</b>: Where the appointment takes place (because it is in Brazil, I don't think we can know the difference between each neighbour, so I might delete this column)
>><br><b>Scholarship</b>: True of false. This is a broad topic, you can read this article --> [Link](https://en.wikipedia.org/wiki/Bolsa_Fam%C3%ADlia)
>><br><b>Hypertension</b>: True or False.
>><br><b>Diabetes</b>: True or false.
>><br><b>Alcoholism</b>: True or False.
>><br><b>Handicap</b>: True or False.
>><br><b>SMS_received</b>: 1 or more messages sent to the patient.
>><br><b>No-Show</b>: 1, who’s didn’t show. 0, who did show in his appointment.




### Question(s) for Analysis
>



In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import parser
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling


### General Properties

In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.
database = "EDA_pro.db"

conn = sqlite3.connect(database)

df = pd.read_sql_query("select * from noshowappointments", con=conn)

conn.close()

In [ ]:
df.head(1)

In [ ]:
df.info()

### Data Cleaning
>first things first: I will remove some columns that we do not need them in my EDA.
><br>such as: <b>patient_id, appointment_id</b>
><br>I will remove <b>neighbourhood</b> because I do not know the difference between eatch neighbourhood.
 

In [ ]:
df.drop(['patient_id', 'appointment_id', 'neighbourhood'], axis=1, inplace=True)

In [ ]:
df.head(1)

In [ ]:
df.describe()

#### minimum age is -1 which is not Realistic, so we should handle it

In [ ]:
df = df[df['age']>=0]

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

#### Now, we should comnvert types of columns (scheduled_day, appointment_day) from a string to datatime type to perform some analysis on it.

In [ ]:
df['scheduled_day']

In [ ]:
df['scheduled_day'] = df['scheduled_day'].apply(lambda x: parser.parse(x))

In [ ]:
df['scheduled_day']

In [ ]:
df['appointment_day'] = df['appointment_day'].apply(lambda x: parser.parse(x))

In [ ]:
df['appointment_day']

In [ ]:
df.head(300)

In [ ]:
df['scheduled_day'] = pd.to_datetime(df.scheduled_day).dt.tz_localize(None)
df['appointment_day'] = pd.to_datetime(df.appointment_day).dt.tz_localize(None)

In [ ]:
df.dtypes

In [ ]:
df['scheduled_day']

In [ ]:
#here we see that in appointment_day column we dont have time, so we should deal with scheduled_day also
df['appointment_day']

In [ ]:
# since we dont have a time in appointment_day values, we will remove time from the values in scheduled day column
df['scheduled_day'] = pd.to_datetime(df['scheduled_day']).dt.date
df

In [ ]:
df.info()

In [ ]:
df.scheduled_day = pd.to_datetime(df.scheduled_day)

In [ ]:
df.head(5)

In [ ]:
df['diff_between_dates'] = df['appointment_day'] - df['scheduled_day'] 

In [ ]:
df.head()

In [ ]:
df['diff_between_dates'].astype(str)

In [ ]:
#df['diff_between_dates'].

## Exploratory Data Analysis


### From our data, what is the average of patient (show up, didn't show up)

In [ ]:
df1 = df['no_show'].value_counts()

In [ ]:
df1

In [ ]:
df1 = df1.to_frame()

In [ ]:
all_p = df['no_show'].count()

In [ ]:
df1

In [ ]:
df1['mean']=df1['no_show']/all_p

In [ ]:
df1

In [ ]:
ax = df1['mean'].plot(kind='bar', figsize=(10,6), color=['lightseagreen', 'cadetblue'], fontsize=13);
ax.set_alpha(0.8)
ax.set_title("Average of patients (show up,  Didn’t show up)", fontsize=22)
ax.set_ylabel("Number", fontsize=15);
ax.set_xlabel("show up", fontsize=15);
plt.show()

#### As we see, most of the patient show uo and there are 20% of patients didn’t show up in their appointment, now let’s dig more about them.

### Is there any connection between age and not shot up?

In [ ]:
df2 = df.groupby('no_show')['age'].mean()

In [ ]:
df2

In [ ]:
ax = df2.plot(kind='bar', figsize=(10,6), color=['teal', 'salmon'], fontsize=13);
ax.set_alpha(0.8)
ax.set_title("Average of patients (show up,  Didn’t show up)", fontsize=22)
ax.set_ylabel("Number", fontsize=15);
ax.set_xlabel("show up", fontsize=15);
plt.show()

#### As we can see, the ones who shows in their appointment older (mean=37) than the ones they didn’t shows in their appointment (mean=34).
 
#### we can observe here that older people is most likley to attend the appointment.

### Is the patient with a scholarship have more probability to not show in the appointment?

In [ ]:
df0 = df.groupby('no_show')['scholarship'].mean()

In [ ]:
df0 = df0.to_frame()

In [ ]:
# all_p: Number of all patients
df0['mean_patient'] = df1['no_show']/all_p

In [ ]:
df0

In [ ]:
# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(8, 10))

# Plot the total patients
sns.set_color_codes("pastel")
sns.barplot(x=['No', 'Yes'], y=df0.mean_patient, data=df0, label="Total", color="b")

# Plot the patients with scholarship
sns.set_color_codes("muted")
sns.barplot(x=['No', 'Yes'], y=df0.scholarship, data=df0, label="patient with scholarship", color="b")

# Add a legend and informative axis label
ax.legend(frameon=False);
ax.set_title('patient with a scholarship', fontsize=20)
ax.set_ylabel('Average of patients',fontsize=15)
ax.set_xlabel('No_show',fontsize=15)
sns.despine(right=True, top=True);

#### As we can see, no show up patient with a scholarship (mean=0.11) which is more than 50% of total no show up patients (mean=0.20)

### Is there any connection between not showing up in the appointment and the time between schedule day and appointment day?

In [ ]:
df_t = df.groupby('no_show')['diff_between_dates'].mean()
df_t=df_t.to_frame()
df_t

In [ ]:
sns.barplot(data=df_t, x=, y=df_t.diff_between_dates)